In [1]:
import shutil
import os
%load_ext tensorboard
if os.path.exists('./logs/'):
    shutil.rmtree('./logs/')

In [2]:
import optuna
from ray.tune.search.optuna import OptunaSearch
import pandas as pd
from pytorch_lightning.callbacks import EarlyStopping
from ray import tune
from ray.tune import CLIReporter
from ray.tune.integration.pytorch_lightning import TuneReportCallback, TuneReportCheckpointCallback
from ray.tune.schedulers import ASHAScheduler
from torchmetrics import MeanAbsoluteError, MeanAbsolutePercentageError, MetricCollection
from darts.dataprocessing.transformers import Scaler
from darts.models import NHiTSModel
from darts import TimeSeries
from pytorch_lightning.callbacks import Callback, ModelCheckpoint
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorboard
import os
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

In [3]:
# 步骤1: 加载CSV文件
df = pd.read_csv('EDvisitfile.csv')

# 确保'date'列是DateTime类型
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

'''
# 分割数据集为训练集、验证集和测试集（假设您已经根据时间排序）
train_proportion = 0.5
val_proportion = 0.3
'''
total_length = len(df)
train_end = 2550
val_end = 4011

'''
# Calculate the split indices
total_length = len(df)
train_end = int(total_length * train_proportion)
val_end = train_end + int(total_length * val_proportion)
'''

# Split the DataFrame
train_df = df.iloc[:train_end]
val_df = df.iloc[train_end:val_end]
test_df = df.iloc[val_end:]

# 步骤2: 使用MinMaxScaler缩放数据
# 定义并拟合scaler
scaler = MinMaxScaler()
scaler.fit(train_df[['No']])  # 只用训练数据拟合scaler

# 缩放训练集和验证集
train_df.loc[:, 'No_scaled'] = scaler.transform(train_df[['No']])
val_df.loc[:, 'No_scaled'] = scaler.transform(val_df[['No']])
test_df.loc[:, 'No_scaled'] = scaler.transform(test_df[['No']])  # 用相同的scaler转换测试集以避免数据泄露

# 转换为TimeSeries对象
train_series = TimeSeries.from_dataframe(train_df, value_cols='No_scaled')
val_series = TimeSeries.from_dataframe(val_df, value_cols='No_scaled')
test_series = TimeSeries.from_dataframe(test_df, value_cols='No_scaled')

# 原始数据转换为TimeSeries对象，如果需要
train_series_origin = TimeSeries.from_dataframe(train_df, value_cols='No')
val_series_origin = TimeSeries.from_dataframe(val_df, value_cols='No')
test_series_origin = TimeSeries.from_dataframe(test_df, value_cols='No')

# 选择需要的列创建多变量时间序列(都是one hot coding)
columns = ['Dayoff', 'Mon', 'Tue', 'Wed', 'Thr', 'Fri', 'Sat', 'Sun', 'YearScaled', 'MonthScaled', 'Dayscaled', 'NewYear', '3Lock']
df_multivariate = df[columns]

# 将DataFrame转换为多变量时间序列
ED_covariates = TimeSeries.from_dataframe(df_multivariate)

C:\Users\ian11\AppData\Local\Temp\ipykernel_4052\2320438073.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.loc[:, 'No_scaled'] = scaler.transform(train_df[['No']])
C:\Users\ian11\AppData\Local\Temp\ipykernel_4052\2320438073.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df.loc[:, 'No_scaled'] = scaler.transform(val_df[['No']])
C:\Users\ian11\AppData\Local\Temp\ipykernel_4052\2320438073.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [4]:
class LossLoggingCallback(Callback):
    def __init__(self):
        super().__init__()
        self.val_losses = []  # To store validation losses
        self.train_losses = []  # To store training losses

    def on_validation_epoch_end(self, trainer, pl_module):
        val_loss = trainer.callback_metrics["val_loss"].item()
        self.val_losses.append(val_loss)
        print(f"Epoch {trainer.current_epoch}: val_loss={val_loss}")
        # Updated report call
        train.report({"loss": val_loss})  # Report the validation loss to Ray Train

    def on_train_epoch_end(self, trainer, pl_module, unused=None):
        if "train_loss" in trainer.callback_metrics:
            train_loss = trainer.callback_metrics["train_loss"].item()
            self.train_losses.append(train_loss)
            print(f"Epoch {trainer.current_epoch}: train_loss={train_loss}")
loss_logging_callback = LossLoggingCallback()

In [5]:
def train_model(model_args, callbacks, train, val):
    torch_metrics = MetricCollection([MeanAbsolutePercentageError(), MeanAbsoluteError()])
    
    # Customize the ModelCheckpoint callback
    model_checkpoint_callback = ModelCheckpoint(
        dirpath="checkpoints",
        filename="{epoch}-{val_loss:.2f}",
        every_n_epochs=5,
    )
    
    model = NHiTSModel(
        input_chunk_length=300, 
        output_chunk_length=30, 
        pl_trainer_kwargs={"callbacks": callbacks, "enable_progress_bar": False},
        log_tensorboard=True,
        **model_args)

    model.fit(
    series=[train_series],
    past_covariates=[ED_covariates],
    val_series=[val_series],
    val_past_covariates=[ED_covariates]
    )

In [6]:
config = {
    'num_stacks': tune.randint(1,4), 
    'num_blocks': tune.randint(1,4), 
    'num_layers': tune.randint(1,4), 
    'layer_widths': tune.randint(20,500), 
    'dropout': tune.uniform(0, 0.8),
}

my_stopper = EarlyStopping(
    monitor="val_loss",
    patience=5,
    min_delta=0.0002,
    mode='min',
)


tune_callback = TuneReportCheckpointCallback(
    {
        "loss": "val_loss",
        "MAPE": "val_MeanAbsolutePercentageError",
    },
    on="validation_end",
)



# define the hyperparameter space

reporter = CLIReporter(
    parameter_columns=list(config.keys()),
    metric_columns=["loss", "MAPE", "training_iteration"],
)

optuna_search = OptunaSearch(metric="MAPE", mode="min")

In [7]:
# Run Ray Tune, optimize hyperparameters by minimizing the MAPE on the validation set
num_samples = 50

scheduler = ASHAScheduler(max_t=1000, grace_period=3, reduction_factor=2)

train_fn_with_parameters = tune.with_parameters(
    train_model, callbacks=[my_stopper, tune_callback], train=train_series, val=val_series,
)

analysis = tune.run(
    train_fn_with_parameters,
    #resources_per_trial=resources_per_trial,
    # Using a metric instead of loss allows for
    # comparison between different likelihood or loss functions.
    metric="loss",  # any value in TuneReportCallback.
    mode="min",
    config=config,
    num_samples=num_samples,
    search_alg=optuna_search,
    scheduler=scheduler,
    progress_reporter=reporter,
    name="tune_darts",
)

print("Best hyperparameters found were: ", analysis.best_config)

2024-03-06 22:07:50,326	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2024-03-06 22:07:52,929	INFO tune.py:220 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2024-03-06 22:07:52,931	INFO tune.py:583 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
[I 2024-03-06 22:07:52,946] A new study created in memory with name: optuna


== Status ==
Current time: 2024-03-06 22:07:53 (running for 00:00:00.13)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: None | Iter 6.000: None | Iter 3.000: None
Logical resource usage: 0/8 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:MX150)
Result logdir: C:/Users/ian11/ray_results/tune_darts
Number of trials: 1/50 (1 PENDING)
+----------------------+----------+-------+--------------+--------------+--------------+----------------+-----------+
| Trial name           | status   | loc   |   num_stacks |   num_blocks |   num_layers |   layer_widths |   dropout |
|----------------------+----------+-------+--------------+--------------+--------------+----------------+-----------|
| train_model_eee41d18 | PENDING  |       |            1 |            2 |            3 |            116 |  0.340825 |
+----------------------+----------+-------+--------------+---------

(train_model pid=8760) GPU available: False, used: False
(train_model pid=8760) TPU available: False, using: 0 TPU cores
(train_model pid=8760) IPU available: False, using: 0 IPUs
(train_model pid=8760) HPU available: False, using: 0 HPUs
(train_model pid=8760) 
(train_model pid=8760)   | Name          | Type             | Params
(train_model pid=8760) ---------------------------------------------------
(train_model pid=8760) 0 | criterion     | MSELoss          | 0     
(train_model pid=8760) 1 | train_metrics | MetricCollection | 0     
(train_model pid=8760) 2 | val_metrics   | MetricCollection | 0     
(train_model pid=8760) 3 | stacks        | ModuleList       | 133 K 
(train_model pid=8760) ---------------------------------------------------
(train_model pid=8760) 100 K     Trainable params
(train_model pid=8760) 32.8 K    Non-trainable params
(train_model pid=8760) 133 K     Total params
(train_model pid=8760) 0.532     Total estimated model params size (MB)
(train_model pid=876

== Status ==
Current time: 2024-03-06 22:08:08 (running for 00:00:15.26)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: None | Iter 6.000: None | Iter 3.000: None
Logical resource usage: 2.0/8 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:MX150)
Result logdir: C:/Users/ian11/ray_results/tune_darts
Number of trials: 2/50 (1 PENDING, 1 RUNNING)
+----------------------+----------+----------------+--------------+--------------+--------------+----------------+-----------+
| Trial name           | status   | loc            |   num_stacks |   num_blocks |   num_layers |   layer_widths |   dropout |
|----------------------+----------+----------------+--------------+--------------+--------------+----------------+-----------|
| train_model_eee41d18 | RUNNING  | 127.0.0.1:8760 |            1 |            2 |            3 |            116 |  0.340825 |
| train_model_bdbf

(train_model pid=8760) WARNING:tensorflow:From C:\Users\ian11\anaconda3\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
(train_model pid=8760) 
(train_model pid=8760) C:\Users\ian11\anaconda3\Lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\b\abs_2112s1s0to\croot\pytorch-select_1700158736573\work\torch\csrc\tensor\python_tensor.cpp:453.)
(train_model pid=8760)   _C._set_default_tensor_type(t)


...

== Status ==
Current time: 2024-03-06 22:29:08 (running for 00:21:15.16)
Using AsyncHyperBand: num_stopped=37
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: -0.01207625494567504 | Iter 6.000: -0.012485759740630027 | Iter 3.000: -0.01443094184662965
Logical resource usage: 1.0/8 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:MX150)
Current best trial: 47614251 with loss=0.011636433592935312 and parameters={'num_stacks': 1, 'num_blocks': 2, 'num_layers': 3, 'layer_widths': 410, 'dropout': 0.6008510996521499}
Result logdir: C:/Users/ian11/ray_results/tune_darts
Number of trials: 50/50 (1 RUNNING, 49 TERMINATED)
+----------------------+------------+-----------------+--------------+--------------+--------------+----------------+-----------+-----------+----------------------+
| Trial name           | status     | loc             |   num_stacks |   num_blocks |   num_layers |   layer_widths |   dropout

2024-03-06 22:29:14,071	INFO tune.py:1042 -- Total run time: 1281.14 seconds (1281.03 seconds for the tuning loop).


== Status ==
Current time: 2024-03-06 22:29:14 (running for 00:21:21.05)
Using AsyncHyperBand: num_stopped=38
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: -0.01207625494567504 | Iter 6.000: -0.012518361125726681 | Iter 3.000: -0.01443094184662965
Logical resource usage: 1.0/8 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:MX150)
Current best trial: 47614251 with loss=0.011636433592935312 and parameters={'num_stacks': 1, 'num_blocks': 2, 'num_layers': 3, 'layer_widths': 410, 'dropout': 0.6008510996521499}
Result logdir: C:/Users/ian11/ray_results/tune_darts
Number of trials: 50/50 (50 TERMINATED)
+----------------------+------------+-----------------+--------------+--------------+--------------+----------------+-----------+-----------+----------------------+
| Trial name           | status     | loc             |   num_stacks |   num_blocks |   num_layers |   layer_widths |   dropout |      los

(train_model pid=17032) Checkpoint successfully created at: Checkpoint(filesystem=local, path=C:/Users/ian11/ray_results/tune_darts/train_model_e33fe0d8_47_dropout=0.4238,layer_widths=280,num_blocks=3,num_layers=3,num_stacks=2_2024-03-06_22-23-38/checkpoint_000005)


Best hyperparameters found were:  {'num_stacks': 1, 'num_blocks': 2, 'num_layers': 3, 'layer_widths': 410, 'dropout': 0.6008510996521499}


In [8]:
%tensorboard --logdir /Users/ian11/ray_results/tune_darts/LSTM2

Reusing TensorBoard on port 6007 (pid 18608), started 1 day, 0:02:16 ago. (Use '!kill 18608' to kill it.)